<h3>Custom sequences</h3>

<h4>example 1</h4>

In [1]:
from functools import lru_cache

In [2]:
class Fib:
    def __init__(self, n):
        self.n = n
        
    def __len__(self):
        return self.n
    
    def __getitem__(self, s):
        if isinstance(s, int):
            if s < 0 or s >= self.n:
                raise IndexError
            else:
                return Fib._fib(s)
            
    @staticmethod
    @lru_cache(2**10)
    def _fib(n):
        return 0 if n == 0 else 1 if n == 1 else Fib._fib(n-1) + Fib._fib(n-2)

In [3]:
f = Fib(10)

print(list(f))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]


<br>

In [4]:
# improvement of the function so we can use negative indices and slices
class Fib:
    def __init__(self, n):
        self.n = n
        
    def __len__(self):
        return self.n
    
    def __getitem__(self, s):
        if isinstance(s, int):                  # if s is integer
            if s < 0:
                s = self.n + s
            if s < 0 or s >= self.n:
                raise IndexError
            else:
                return Fib._fib(s)
        elif isinstance(s, slice):              # if s is slice
            start, stop, step = s.indices(self.n)
            rng = range(start, stop, step)
            return [Fib._fib(i) for i in rng]
            
    @staticmethod
    @lru_cache(2**10)
    def _fib(n):
        return 0 if n == 0 else 1 if n == 1 else Fib._fib(n-1) + Fib._fib(n-2)

In [5]:
f2 = Fib(10)

print(list(f2))
print(f2[-1])
print(f2[1:4])

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
34
[1, 1, 2]


<br>
<br>
<h4>example 2</h4>

In [6]:
class MyClass:
    def __init__(self, name):
        self.name = name
        
    def __repr__(self):
        return f'MyClass(name={self.name})'
    
    def __add__(self, other):
        if isinstance(other, MyClass):
            return MyClass(self.name + other.name)
        else:
            raise TypeError('incompatible types')
    
    def __iadd__(self, other):
        if isinstance(other, MyClass):
            self.name += other.name
        else:
            self.name += other
        return self
    
    def __mul__(self, n):
        return MyClass(self.name * n)
    
    def __rmul__(self, n):
        return self.__mul__(n)
    
    def __imul__(self, n):
        self.name *= n
        return self
    
    def __contains__(self, value):
        return value in self.name

In [7]:
c1 = MyClass('Eric_')
c2 = MyClass('Idle_')
print(id(c1))
print(id(c2))

139934133715520
139934133715128


In [8]:
result_add = c1 + c2
print(id(result_add), result_add)

139934133715800 MyClass(name=Eric_Idle_)


In [9]:
resul_mult = c1 *3
print(id(resul_mult), resul_mult)

139934133716472 MyClass(name=Eric_Eric_Eric_)


In [10]:
print('Eric' in c1)
print('r' in c1)
print('z' in c1)

True
True
False


In [11]:
c1 += c2
print(id(c1), c1)

139934133715520 MyClass(name=Eric_Idle_)


In [12]:
c1 *= 2
print(id(c1), c1)

139934133715520 MyClass(name=Eric_Idle_Eric_Idle_)


<br>
<br>
<h4>example 3</h4>

In [13]:
import numbers

In [14]:
class Point:
    def __init__(self, x, y):
        if isinstance(x, numbers.Real) and isinstance(y, numbers.Real):
            self._pt = (x, y)
        else:
            raise TypeError('Point co-ordinates must be real numbers.')
            
    def __repr__(self):
        return f'Point(x={self._pt[0]}, y={self._pt[1]})'
    
    def __len__(self):
        return len(self._pt)
    
    def __getitem__(self, s):  # used for unpacking
        return self._pt[s]

In [15]:
p1 = Point(10, 2)
p1

Point(x=10, y=2)

In [16]:
x, y = p1
x, y

(10, 2)

In [17]:
p2 = Point(*p1)
print(id(p1), p1)
print(id(p2), p2)

139934133810232 Point(x=10, y=2)
139934134189416 Point(x=10, y=2)


In [18]:
class Polygon:
    def __init__(self, *pts):
        self._pts = [Point(*pt) for pt in pts]

    def __repr__(self):
        pts_str = ', '.join([str(pt) for pt in self._pts])
        return f'Polygon({pts_str})'
    
    def __len__(self):
        return len(self._pts)
    
    def __getitem__(self, s):
        return self._pts[s]
    
    def __setitem__(self, s: slice, value):
        try:
            rhs = [Point(*pt) for pt in value]
            is_single = False
        except TypeError:
            try:
                rhs = Point(*value)
                is_single = True
            except TypeError:
                raise TypeError('Invalid Point of iterable of Points')
                
        if (isinstance(s, int) and is_single) \
            or (isinstance(s, slice) and not is_single):
                self._pts[s] = rhs
        else:
            raise TypeError('Incompatible index/slice assignment')
    
    def __add__(self, other):
        if isinstance(other, Polygon):
            new_pts = self._pts + other._pts
            return Polygon(*new_pts)
        else:
            raise TypeError('can only concatenate with another Polygon')
    
    def append(self, pt):
        self._pts.append(Point(*pt))
        
    def insert(self, i, pt):
        self._pts.insert(i, Point(*pt))
        
    def extend(self, pts):
        if isinstance(pts, Polygon):
            self._pts += pts._pts
        else:
            points = [Point(*pt) for pt in pts]
            self._pts += points
            
    def __iadd__(self, other):
        self.extend(other)
        return self
    
    def __delitem__(self, s):
        del self._pts[s]
        
    def pop(self, i):
        return self._pts.pop(i)
    
    def clear(self):
        self._pts.clear()

In [19]:
pgn = Polygon((0,0), Point(1,1))
print(id(pgn), pgn)

139934133885360 Polygon(Point(x=0, y=0), Point(x=1, y=1))


In [20]:
pgn2 = Polygon(Point(x=0, y=0), Point(x=1, y=1))
print(id(pgn2), pgn2)

139934133846200 Polygon(Point(x=0, y=0), Point(x=1, y=1))


In [21]:
len(pgn)

2

In [22]:
pgn[::-1]

[Point(x=1, y=1), Point(x=0, y=0)]

In [23]:
pgn1 = Polygon((0,0), (1,1))
pgn2 = Polygon((2,2), (3,3))

pgn_add = pgn1 + pgn2
pgn_add

Polygon(Point(x=0, y=0), Point(x=1, y=1), Point(x=2, y=2), Point(x=3, y=3))

In [24]:
pgn1.append([10, 10])
pgn1

Polygon(Point(x=0, y=0), Point(x=1, y=1), Point(x=10, y=10))

In [25]:
pgn1.insert(1, Point(-1, -1))
pgn1

Polygon(Point(x=0, y=0), Point(x=-1, y=-1), Point(x=1, y=1), Point(x=10, y=10))

In [26]:
pgn1.extend([(20, 20), Point(21, 21)])
pgn1

Polygon(Point(x=0, y=0), Point(x=-1, y=-1), Point(x=1, y=1), Point(x=10, y=10), Point(x=20, y=20), Point(x=21, y=21))

In [27]:
pgn1[1:3]

[Point(x=-1, y=-1), Point(x=1, y=1)]

In [28]:
pgn1[1:3] = [(51,51), Point(52,52), [53,53]]
pgn1

Polygon(Point(x=0, y=0), Point(x=51, y=51), Point(x=52, y=52), Point(x=53, y=53), Point(x=10, y=10), Point(x=20, y=20), Point(x=21, y=21))

In [29]:
pgn1[-1] = Point(40,40)
pgn1

Polygon(Point(x=0, y=0), Point(x=51, y=51), Point(x=52, y=52), Point(x=53, y=53), Point(x=10, y=10), Point(x=20, y=20), Point(x=40, y=40))

In [30]:
del pgn1[-1]
pgn1

Polygon(Point(x=0, y=0), Point(x=51, y=51), Point(x=52, y=52), Point(x=53, y=53), Point(x=10, y=10), Point(x=20, y=20))

In [31]:
print(pgn1.pop(1))
print(pgn1)

Point(x=51, y=51)
Polygon(Point(x=0, y=0), Point(x=52, y=52), Point(x=53, y=53), Point(x=10, y=10), Point(x=20, y=20))


In [32]:
pgn1.clear()
pgn1

Polygon()